In [5]:
import random
import torch
import numpy as np
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [5]:
!pip install  efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16033 sha256=04b98da8555700910acb4cc5822ff46ef8fdc091003463ddd476da4b63bbc074
  Stored in directory: /root/.cache/pip/wheels/f6/09/44/bec839054285e5cee64da8d7696e462f7dad0787fdde0a358c
Successfully built efficientnet-pytorch
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
import numpy as np
# You may add any imports you need
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt
import glob
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize, Normalize, ToTensor, Compose, RandomHorizontalFlip, RandomVerticalFlip, RandomAffine,RandomRotation
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import  models
from efficientnet_pytorch import EfficientNet

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform):
        # YOUR CODE
        self.dataset = ImageFolder(root=data_dir, transform=transform)
    def __getitem__(self, idx):
        # YOUR CODE
        return self.dataset[idx]
    def __len__(self, idx=None):
        # YOUR CODE
        return len(self.dataset)

In [ ]:
# !curl -L $(yadisk-direct https://yadi.sk/d/dI8NrPZAugqtVg) -o dataset.zip #скачаем
# !unzip /content/drive/MyDrive/dataset.zip > /dev/null  # распакуем
# !rm dataset.zip # удалим 

In [14]:
!unzip dataset.zip > /dev/null  # распакуем

In [7]:

width = 224
height = 224

train_transform = Compose([Resize((width,height),interpolation=Image.BILINEAR), ToTensor(), 
                           Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                           RandomHorizontalFlip(), RandomRotation(45)])
val_transform = Compose([Resize((width, height), interpolation=Image.BILINEAR), ToTensor(), Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), ])

train_dataset = MyDataset("./dataset/dataset/train", transform=train_transform)
val_dataset = MyDataset("./dataset/dataset/val", transform=val_transform)


In [8]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score

def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:0"):
    model.to(device).train()
    losses = []
    with tqdm(total=len(train_dataloader)) as pbar:
        for imgs, labels in train_dataloader:
            labels = labels.to(device)
            imgs = imgs.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            _, predicted = torch.max(outputs.detach(), 1)
            accuracy = accuracy_score(predicted.cpu(), labels.cpu())
            pbar.set_description('Loss: {:.4f}; Accuracy: {:.4f}%'.format(loss.item(), 100*accuracy))    
            pbar.update(1)

def predict(model, val_dataloder, criterion, device="cuda:0"):
    model.to(device).eval()
    losses = []
    predicted_classes = []
    true_classes = []
    with tqdm(total=len(val_dataloader)) as pbar:
        with torch.no_grad():
            for imgs, labels in val_dataloader:
                labels = labels.to(device)
                imgs = imgs.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)

                losses.append(loss.item())
                _, predicted = torch.max(outputs, 1)
                predicted_classes.append(predicted)
                true_classes.append(labels)
                accuracy = accuracy_score(predicted.cpu(), labels.cpu())

                pbar.set_description('Loss: {:.4f}; Accuracy: {:.4f}%'.format(loss.item(), 100*accuracy))
                pbar.update(1)

    predicted_classes = torch.cat(predicted_classes).detach().to('cpu').numpy()
    true_classes = torch.cat(true_classes).detach().to('cpu').numpy()
    return losses, predicted_classes, true_classes


def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10, scheduler=None):
    model.to(device)
    lrs = []
    for epoch in range(n_epochs):
        print('Learning rate: ', optimizer.param_groups[0]['lr'])
        print('Epoc:', epoch)
        train_one_epoch(model, train_dataloader, criterion, optimizer)
        print('Validation')
        losses, predicted_classes, true_classes = predict(model, val_dataloader, criterion)
        print('Accuracy: ', accuracy_score(true_classes, predicted_classes))
        lrs.append(optimizer.param_groups[0]['lr'])
        scheduler.step()
    plt.plot(lrs)

In [9]:
model_resnet = models.resnet101(pretrained=True)
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 200)
model_resnet.load_state_dict(torch.load('resnet101_8313.pth'))

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


<All keys matched successfully>

In [ ]:
losses, predicted_classes, true_classes = predict(model_resnet, val_dataloader, criterion)
print('Accuracy: ', accuracy_score(true_classes, predicted_classes))

In [12]:
model_efient = EfficientNet.from_pretrained('efficientnet-b7')
model_efient.load_state_dict(torch.load('efient_8492.pth'))

Loaded pretrained weights for efficientnet-b7


<All keys matched successfully>

In [ ]:
losses, predicted_classes, true_classes = predict(model_efient, val_dataloader, criterion)
print('Accuracy: ', accuracy_score(true_classes, predicted_classes))

Возьмем веса моделей, которые я ранее смог дообучить и соединим их вместе, чтобы получить нужный результат. К сожалению, этапы обучения не сохранились, но они были долгие и мучительные, решающую  роль сыграл schedular. Много перепробовал идей, но хорошо на этом датасете обучается резнет, только вот он в большинстве своем доходил до 82..83 и дальше вообще не хотел идти. 
Для EfficientNet ключевую роль сыграли аугментации..я не помню но кажется он даже после первой эпохи выдал шокирующее 83, но потом обучался ооооооооочень медленно. 
Еще есть такая тема, как Label Smoothing Cross Entropy, но к сожалению я ее не успел попробовать, в теории с ней accuracy должно подрасти. Дело в том, что у нас есть проблема, в том что сеть может быть самоуверенной,по сути  Label Smoothing это метод регуляризации, который решает данную проблему.

In [13]:
class mixmodels(nn.Module):
    def __init__(self, modelA, modelB, in_features=400, h1=300, d=0.35, num_classes=10):
        super(mixmodels, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        # Remove last linear layer
        self.modelA._fc = nn.Identity()
        self.modelB.fc = nn.Identity()

        self.drop1 = nn.Dropout(d)
        self.fc1 = nn.Linear(in_features, 1024)
        self.relu1 = nn.ReLU(inplace=True)

        self.drop2 = nn.Dropout(d)
        self.fc2 = nn.Linear(1024, 1024)
        self.relu2 = nn.ReLU(inplace=True)

        self.fc3 = nn.Linear(1024, num_classes)
        
        
    def forward(self, x):
        x1 = self.modelA(x.clone())  # clone to make sure x is not changed by inplace methods
        x1 = x1.view(x1.size(0), -1)
        x2 = self.modelB(x)
        x2 = x2.view(x2.size(0), -1)
        x = torch.cat((x1, x2), dim=1)

        x = self.drop1(x)
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.drop2(x)
        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)

        return x

In [14]:
for param in model_resnet.parameters():
    param.requires_grad_(False)

for param in model_efient.parameters():
    param.requires_grad_(False)

In [16]:
in_features = model_efient._fc.in_features + model_resnet.fc.in_features

In [17]:
model = mixmodels(model_efient, model_resnet, in_features=in_features, h1=1024, d=0.45, num_classes=200 )

In [19]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=256, num_workers=4)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2, eta_min=0.00001, last_epoch=-1)
n_epochs = 50

In [20]:
train(model, train_dataloader, val_dataloader, criterion, optimizer, device, n_epochs, scheduler)

  0%|          | 0/391 [00:00<?, ?it/s]

Learning rate:  0.1
Epoc: 0


  0%|          | 0/40 [00:00<?, ?it/s]

Validation


  0%|          | 0/391 [00:00<?, ?it/s]

...

Validation


  0%|          | 0/391 [00:00<?, ?it/s]

Accuracy:  0.8606
Learning rate:  0.009558195366224508
Epoc: 13


Loss: 0.5032; Accuracy: 86.3281%:   1%|          | 2/391 [00:05<18:52,  2.91s/it]


KeyboardInterrupt: 

In [24]:
losses, predicted_classes, true_classes = predict(model, val_dataloader, criterion)
accuracy = accuracy_score(true_classes, predicted_classes)
print('Accuracy: ', accuracy)

Loss: 0.4793; Accuracy: 87.5000%: 100%|██████████| 40/40 [00:52<00:00,  1.31s/it]

Accuracy:  0.8603


In [25]:
etalon = 0.86
if accuracy >= etalon:
    print("ПОБЕДА!!!!")
else:
    print('обучай салага дальше')

ПОБЕДА!!!!
